In [1]:
import torch
import numpy as np
from dexnet.grasp_model import DexNetBase as Model
from concurrent.futures import ThreadPoolExecutor

import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import transforms

In [2]:
key = "depth_ims_tf_table"
# dataset_path = "dataset/dexnet_3/dexnet_09_13_17"
dataset_path = "dataset/dexnet_2/dexnet_2_tensor"
def key_to_npz(key, idx: int) -> str:
    return f"{dataset_path}/tensors/{key}_{str(idx).zfill(5)}.npz"

def loader(idx):
    with np.load(key_to_npz(key, idx)) as x:
        arr = x["arr_0"].astype(np.float16)
    return arr

with ThreadPoolExecutor(max_workers=8) as executor:
    data = executor.map(
        lambda idx: loader(idx),
        range(6000),
    )
data = torch.from_numpy(np.concatenate(list(data)))

In [29]:
print(torch.mean(data.double()))
print(torch.min(data))

tensor(0.7002, dtype=torch.float64)
tensor(0.4448, dtype=torch.float16)


In [3]:
print(data.shape)

torch.Size([6000000, 32, 32, 1])


In [3]:
model = Model()
model.load_state_dict(torch.load("outputs/DexNet3/best_valid_dex3_adam.pth"))
model = model.to("cuda:6")

In [4]:
normalizers = (
    0.59784445,
    0.00770147890625,
)  # mean, std value for depth images (from analyze.py script)
def preprocess(
    img: torch.Tensor
) -> torch.Tensor:
    """
    Preprocesses the image for training.
    """
    assert len(img.shape) == 3, "Image shape must be (H, W, C)."

    img = img.permute(2, 0, 1).float()  # (H, W, C) -> (C, H, W)

    # Normalize image data.
    img = (img - normalizers[0]) / normalizers[1]

    # Create noise -- with same shape as image.
    gp_noise = torch.randn(8, 8) * 0.005
    gp_noise = F.interpolate(
        gp_noise.unsqueeze(0).unsqueeze(0), scale_factor=4.0, mode="bicubic"
    ).squeeze()
    assert gp_noise.shape[-1] == img.shape[-1], "Noise shape mismatch."

    # Add the noise to the image where pixel values are greater than 0
    mask = (img > 0).float()
    img = img + gp_noise * mask

    # Image augmentation.
    if np.random.rand() < 0.5:
        img = transforms.functional.rotate(img, 180)
    if np.random.rand() < 0.5:
        img = transforms.functional.vflip(img)
    if np.random.rand() < 0.5:
        img = transforms.functional.hflip(img)

    return img

In [5]:
imgs = torch.stack([preprocess(data[i]) for i in range(1000)])
print(imgs.dtype)
imgs = imgs.to("cuda:6")
#img = preprocess(data[0]).to("cuda").unsqueeze(0)
print(imgs.shape)
outs = model(imgs)
print(outs.shape)
print(outs[torch.where(outs > .05)])

torch.float32
torch.Size([1000, 1, 32, 32])
torch.Size([1000, 1])
tensor([0.0638, 0.0676, 0.0864, 0.1604, 0.0987, 0.0962, 0.6596, 0.1091, 0.5848,
        0.1946, 0.3608, 0.1715, 0.1245, 0.0540, 0.0693, 0.1508, 0.0704, 0.1144,
        0.1051, 0.0625, 0.0773, 0.0549, 0.0870, 0.3560, 0.0510, 0.0662, 0.1877,
        0.1542, 0.1609, 0.0584, 0.1804, 0.1159, 0.1149, 0.2455, 0.0857, 0.0814,
        0.1913, 0.2606, 0.0543, 0.2112, 0.0993, 0.1621, 0.2595, 0.0976, 0.3142,
        0.2071, 0.2298, 0.5688, 0.2130, 0.0677, 0.3146, 0.3189, 0.4037, 0.1530,
        0.0986, 0.1044, 0.3185, 0.2094, 0.0586, 0.3077, 0.0628, 0.0588, 0.0541,
        0.0939, 0.2107, 0.3289, 0.0991, 0.1869, 0.1643, 0.4445, 0.0683, 0.0965,
        0.2443, 0.0659, 0.8409, 0.1240, 0.0681, 0.3666, 0.6636, 0.0584, 0.1174,
        0.0676, 0.1360, 0.3816, 0.2341, 0.1305, 0.2093, 0.3798, 0.1658, 0.3096,
        0.0510, 0.1491, 0.2930, 0.2335, 0.1743, 0.0916, 0.0892, 0.5976, 0.6298,
        0.0517, 0.5319, 0.1144, 0.1048, 0.2014, 0.4923

In [6]:
key = "robust_suction_wrench_resistance"
with ThreadPoolExecutor(max_workers=8) as executor:
    metrics = executor.map(
        lambda idx: loader(idx),
        range(2),
    )
metrics = torch.from_numpy(np.concatenate(list(metrics))).float()
metrics = torch.stack([metrics[i] for i in range(1000)])
metrics.dtype

torch.float32

In [7]:
metrics = metrics.to("cpu")
outs = outs.cpu()
print(outs[torch.where(metrics > .1)])

tensor([[0.0039],
        [0.0127],
        [0.0676],
        [0.0987],
        [0.0220],
        [0.1946],
        [0.1245],
        [0.0540],
        [0.1508],
        [0.0404],
        [0.0125],
        [0.0072],
        [0.0339],
        [0.0022],
        [0.0048],
        [0.0784],
        [0.0699],
        [0.1520],
        [0.3895],
        [0.1051],
        [0.4286],
        [0.4629],
        [0.4315],
        [0.1159],
        [0.0190],
        [0.1966],
        [0.0322],
        [0.1324],
        [0.0166],
        [0.1160],
        [0.0689],
        [0.0059],
        [0.1931],
        [0.2004],
        [0.3602],
        [0.0604],
        [0.1021],
        [0.1852],
        [0.0358],
        [0.0588],
        [0.2749],
        [0.0744],
        [0.3781],
        [0.0148],
        [0.5073],
        [0.0070],
        [0.3157],
        [0.0079],
        [0.0482],
        [0.0715],
        [0.5756],
        [0.5451],
        [0.3031],
        [0.1338],
        [0.0471],
        [0

In [22]:
gt_thresh = .2
thresh = .01
print(metrics.shape)
print(outs.shape)
metrics = metrics.reshape(1000, 1)
def getPrecisionRecall(outputs, wrench_resistances, gt_thresh, thresh=0.2):
    tp = ((outputs > thresh) & (wrench_resistances > gt_thresh)).sum().item()
    fp = ((outputs > thresh) & (wrench_resistances <= gt_thresh)).sum().item()
    fn = ((outputs <= thresh) & (wrench_resistances > gt_thresh)).sum().item()

    return tp, fp, fn
tp, fp, fn = getPrecisionRecall(
    outs, metrics, gt_thresh, thresh=thresh
)
print(tp, fp, fn)

torch.Size([1000, 1])
torch.Size([1000, 1])
69 293 7
